In [1]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
# Following pagination links
class QuoteMultipleSpider(scrapy.Spider):
    name = "quotesmultiplepages"
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
    ]
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield{
                'text': quote.css('span.text::text').get(),
                'author' :quote.css('span small::text').get(),
                'tags' :quote.css('div.tags a.tag::text').getall(),
            }
        try:
            # select the NEXT button and store it in next page
            next_page = response.css('li.next a').attrib["href"]
        except KeyError:
            # In the last page, there won't be any "href" and a KeyError will be raised
            logging.info('No next page. Terminating crawling process.')
        else:
            # If a next page is found, execute the parse method once again
            yield response.follow(next_page, callback=self.parse)

In [3]:
filename = "6_quotesautothrottle.json"

if filename in os.listdir('src/'):
        os.remove('src/' + filename)

process = CrawlerProcess(settings={
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'LOG_LEVEL': logging.INFO,
        "FEEDS":{
                'src/' + filename : {"format":"json"},
        },
       "AUTOTHROTTLE_ENABLED": True
})

# Start the crawling using the spider you defined above
process.crawl(QuoteMultipleSpider)
process.start()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3037677083.py, line 9)